# Estimación del Egreso de Estudiantes de Ingeniería en función de los créditos acumulados a los tres años de su ingreso 
<h2> Generaciones (2000 - 2005)</h2>

<h2>Tabla de Contenido</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="obj">Objetivo</a></li>
        <li><a href="conex">Conexión a la Base de Datos</a></li>
        <li><a href="list">Como listar las Tablas y su Esquema en una Base SQLite3</a></li>
        <li><a href="query">Formulación de Consultas SQL</a></li>
        <li><a href="data">Preparando los Datos de Entrada</a></li>
        <li><a href="model">Creación del modelo de Regresión Logística</a></li>
        <li><a href="eval">Evaluación de la Performance del Regresor</a></li>
        <li><a href="fin">Conclusiones</a></li>
    </ul>
    <p>
        Tiempo estimado: <strong>90 min</strong>
    </p>
</div>

<hr>

<h2 id=obj>Objetivo</h2>

En este notebook desarrollaremos un modelo de regresion logistica para estimar la probabilidad de egreso de estudiantes de las carreras de Ingeniería Civil, Mecánica, Producción, Naval, Eléctrica y Computación.
La idea aqui es concetar con una base de datos SQLite 3 con datos de estudiantes de las generaciones 2000 a 2005 inclusive.
Se generará una consulta SQL para extraer los datos de las tablas de la base y luego se importaran los datos en un dataframe.
La idea del trabajo es predecir si un estudiante egresara o no en función de los creditos ganados a los tres años de ingresos. 
Teóricamente un estudiante ideal deberia avanzar ganando 75 creditos al año para estar siempre al día y poder egresar a los 5 años. En la realidad los estudiantes egresan en poco mas de 8 años en promedio según datos de eficiencia de egreso 2018.

In [ ]:
# Cargo los paquetes por defecto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


<h2 id=conex>Conexión a la Base de Datos</h2>

Ahora creamos una conexion a la base de datos <code>ingenieria.db</code>.

**Nota:** Aqui tenemos que tener presente donde tenemos el archivo ingenieria.db porque pueden dar un path absoluto  o relativo al archivo. Para saber que notacion uisar les dejo la url de la seccion referente a database paths notation en el manual de create_engine().

https://docs.sqlalchemy.org/en/13/core/engines.html#database-urls

Yo voy a asumir que tengo el archivo <code>ingenieria.db</code> en el mismo directorio de trabajo del notebook. asi que usare tres slashes ///


In [ ]:
from sqlalchemy import create_engine, inspect

# Reemplaza 'tu_base_de_datos.db' con la ruta a tu archivo SQLite
engine = create_engine('sqlite:///ingenieria.db?charset=utf8')


<h2 id=list>Como listar las Tablas y su Esquema en una Base SQLite3</h2>

Una vez creada la conexion es necesario saber que tablas hay y como es el esquema de cada una para poder realizar consultas. Esto se hace a travez del <code>inspector</code>
En la celda de abajo se da el codigo para realizar estas dos tareas.

In [ ]:
# Para ver las tablas que hay en la base

inspector = inspect(engine)

table_names = inspector.get_table_names()


possible_encodings = ['latin-1', 'cp1252', 'iso-8859-15']


print("Tablas en la base de datos:")
for table_name in table_names:
    print(f"- {table_name}")
    
    


In [ ]:
# Cierro la conexion y libero el recurso
engine.dispose()

Esta base tiene algunos caracteres no UTF-8 asi que puede dar error en cualquier nombre de campo, tabla o registro (diresis, tildes, apostrofes etc, todo eso da error si no se decodifica correctamente)
Para eso casos pueden probar algo como esto.


In [ ]:
# creo una funcion para lidiar con las distintas codificaciones que pueda haber

def safe_decode(text, encodings):
    for enc in encodings:
        try:
            return text.encode(enc).decode('utf-8')
        except UnicodeEncodeError:
            pass
        except UnicodeDecodeError:
            pass
    return text  # Si no se puede decodificar, devuelve el texto original



print("Nombres de las tablas:")

for table_name in table_names:
    decoded_name = safe_decode(table_name, possible_encodings)
    print(f"- Original: {table_name}, Decodificado: {decoded_name}")
    columns = inspector.get_columns(table_name)
    for column in columns:
        column_name = column['name']
        decoded_column_name = safe_decode(column_name, possible_encodings)
        print(f"  - Original: {column_name}, Decodificado: {decoded_column_name}")

In [ ]:
# Para saber el esquema de las tablas
#coding: utf-8

print("Esquemas de las tablas:")
for table_name in table_names:
    print(f"\nEsquema de la tabla: {table_name}")
    columns = inspector.get_columns(table_name)
    for column in columns:
        print(f"  - Nombre: {column['name']}")
        print(f"    Tipo: {column['type']}")
        print(f"    Nulable: {column['nullable']}")
        print(f"    Clave Primaria: {column['primary_key']}")
        if 'default' in column:
            print(f"    Valor por defecto: {column['default']}")

<h2 id=query>Formulación de Consultas SQL</h2>

Despues de inspeccionar las tablas de la base y sus relaciones, estamos en condiciones de crear un par de consultas SQL a la Base de Datos para definir el dataset inicial de trabajo.

1) Listado de Alumnos que ingresaron a las carreras de ingenieria Civil, Mecánica, Naval, Eléctrica , Producción y Computación entre 2000 y 2005 inclusive.

2) Hallar los créditos acumulados a los 2.5 años de ingreso para cada uno

3) Crear un Dataframe que contenga las siguientes columnas: Id, Carrera, Ciclo, Generacion, Creds2a, FechaEgreso

Esta son consultas complejas porque la vieja base del SGB de Ingeniería era muy desordenada y requería agregar diversas condiciones para filtrar casos atípicos. Por esta razón yo les proporcionaré las consultas ya preparadas. 


In [ ]:
# Lleno un tabla temporal con las cedulas de los ingresos entre 2005 y 2009 a las carreras en cuestion
# aqui se tamizan los casos atipicos, cambios de plan etc. Queremos ingresos puros no estudiantes
# de generaciones anteriores

with engine.connect() as conn:
    rs = conn.execute('delete from temporal')
    conn.close()

query1 = 'insert into temporal select distinct cedula from estudcarr where generacion between 2005 and 2009 \
        and carrera in (22, 72)' 


Chequeamos que la tabla temporal se haya poblado

In [ ]:
# Lleno la tabla tremporal con las cedulas deseadas
# Vean que aqui estoy haciendo una operacion en la propia base de datos asi que no tengo necesidad de extraer el
# resultado de la operacion a un dataframe. Por eso uso la sintaxis tradicional con `with`
with engine.connect() as conn:
    rs = conn.execute(query1)
    conn.close()


In [ ]:
# Chequeo que los datos hayan sido cargados y saco alguna info basica del nro de registros, tipos etc.

df = pd.read_sql_query('select * from temporal', engine)

df.info()   

Una vez preparados los datos de las generaciones hago un aconsulta para saber los créditos a dos años de haber empezado cada alumno de cada carrera. Pueden ver que no hay inner joins indicados, PERO si es un inner join de tres tablas. En la notacion explicita deberia haber invocado inner join cada dos tablas que cruzo, pero porque no lo hice aqui?

La consulta anidada se debe a que hay alumnos que hacen mas de una carrera (para saltearse las previaturas), entonces convenimos en elegir la carrera y ciclo donde tienen la mayor cantidad de creditos ganados. La idea es estudiar estudiantes "puros" o sea que hagan una sola carrera para reducir sesgos.

In [ ]:
# Saco la suma de créditos a los tres años de haber empezado.

query2 = 'select * from (select s.cedula as "cedula", u.carrera as "carrera", u.ciclo as "ciclo", sum(creditos) as "Creds3a", \
Fechaing from activ2 s, asigcarr t, estudcarr u \
where s.cedula = u.cedula and t.carrera = u.carrera and t.ciclo = u.ciclo and t.carrera in (22, 72) and \
s.asignatura= t.asignatura and tipoactividad in ("E", "R") and nota > 2 and fecha between Fechaing and \
Fechaing + 30300 and s.cedula in temporal \
group by u.cedula, u.carrera, u.ciclo) group by cedula having(max(Creds3a))'


df = pd.read_sql_query(query2, engine)


Inspeccionamos la salida de la consulta

In [ ]:
# Vemos las primeras 10 filas
df.head(10)

In [ ]:
# obtenemos informacion sobre los creditos
df.info()

Podemos crear un histograma con la ditribucion de creditos, lo saco mediante Matplotlib y mediante Seaborn
vean las diferencias de cada notacion.

In [ ]:
plt.hist(df['Creds3a'], bins=45, range=(0, 350))
plt.show()

In [ ]:
# Una version mas linda del histograma con seaborn

sns.histplot(df['Creds3a'], bins = 45, binrange=(0, 350))

In [ ]:
# Saco estadisticas del dataset
df.describe()

#### Exploración de los Datos Obtenidos

Podemos realizar un EDA basico mediante un boxplot de la salida de la consulta por carrera y ciclo. Como carrera y ciclo son dos categorias dferentes y tengo que agrupar en la combinacion de ambas debo crear una nva columna en el Dataframe llamada carrera-ciclo y agrupar sobre ella.

In [ ]:
# creo la columna carrera-ciclo
df['carrera_ciclo'] = df['carrera'].astype(str) + '-' + df['ciclo'].astype(str)


sns.boxplot(data=df, x="carrera_ciclo", y="Creds3a")

Hacemos los mismo pero con la foto a 6 años de Ingresado, para control

In [ ]:
query3 = 'select * from (select s.cedula as "cedula", u.carrera as "carrera", u.ciclo as "ciclo", \
sum(creditos) as "Creds6a", Fechaing, Fechaegr from activ2 s, asigcarr t, estudcarr u \
where s.cedula = u.cedula and t.carrera = u.carrera and t.ciclo = u.ciclo and t.carrera in (22, 72) and \
s.asignatura= t.asignatura and tipoactividad in ("E", "R") and nota > 2 and fecha between Fechaing and \
Fechaing + 60300 and s.cedula in temporal \
group by u.cedula, u.carrera, u.ciclo) group by cedula having(max(Creds6a))'


df2 = pd.read_sql_query(query3, engine)

In [ ]:
# obtenemos informacion sobre los creditos
df2.info()

Visualizamos los resultados
con otro histograma y boxplot

In [ ]:
# Histograma de creditos a los 6 anios de ingreso

sns.histplot(df2['Creds6a'], bins = 45, binrange=(0, 450))

In [ ]:
# creo la columna carrera-ciclo en Df2
df2['carrera_ciclo'] = df2['carrera'].astype(str) + '-' + df2['ciclo'].astype(str)


sns.boxplot(data=df2, x="carrera_ciclo", y="Creds6a")

<h2 id=data>Preparando los Datos de Entrada</h2>


Ahora debo realizar una ultima consulta para ver quienes egresaron a los 6 anios de las generaciones involucradas

In [ ]:
df2.head()



In [ ]:
df2.info()

Debo cambiar el tipo de la columna Fechaegr a "int32" porque aparece como "object" o sea como strings. Ademas debo recodificar Fechaegr como: 0 - No egresado, 1- egresado usando label encoding. 

In [ ]:
df2.replace({'Fechaegr': ""}, {'Fechaegr': '0'}, inplace=True)


In [ ]:
df2['Fechaegr']=df2['Fechaegr'].astype('int32')

In [ ]:
df2.head()

In [ ]:
df2.loc[df2['Fechaegr'] > 0, 'Fechaegr']=1 

In [ ]:
df2.head(20)

In [ ]:
# Renombro la columna "Fechaegr" a "Egresa"
df2.rename(columns={'Fechaegr': 'Egresa'}, inplace=True)

In [ ]:
df2.head(16)

Preparo el dataset de trabajo: concateno df2 y df['Creds3a'] como entrada para el modelo. Tb debo agregar ceros en los casos de alumnos que no obtuvieron creditos en los primeros 3 años y que luego a los 6 ya tienen algo.  

In [ ]:
df.set_index('cedula', inplace=True)

In [ ]:
df2.set_index('cedula', inplace=True)

In [ ]:
df2.head()

In [ ]:
data_ing = pd.concat([df2, df['Creds3a']], axis=1)

In [ ]:
data_ing.head(30)



En el caso de alumnos que no generaron creditos en los primeros 3 anios el concatenamiento les asigno NaN por defecto en la columna 'Creds3a', debemos reemplazar los NaN por un valor numerico (0 en este caso)

In [ ]:
data_ing.replace({'Creds3a': np.NaN}, {'Creds3a': 0}, inplace=True)

In [ ]:
data_ing.head(20)

Tambien debo cambiar el tipo de los datos de la columna "Creds3a" de "object" a "int32"

In [ ]:
data_ing['Creds3a']=data_ing['Creds3a'].astype('int64')

In [ ]:
# Checo los tipos de las columnas en el dataframe de trabajo.
data_ing.info()

<h2 id=model>Creación del modelo de Regresión Logística</h2>

#### - Identificamos la columna objetivo y las caracteristicas

Primero Identificamos la variable objetivo (target)

In [ ]:
target = data_ing['Egresa']

In [ ]:
features=data_ing[['carrera', 'ciclo', 'Creds3a', 'Creds6a']]

In [ ]:
#Importamos la biblioteca sklearn y las funcionalidades de regresion logistica
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


In [ ]:
X = features.values
y = target.values


#1- Dataset Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("Original shape: ", X_train.shape)



In [ ]:
# Create logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

#4- Make predictions on the test dataset.
y_pred = model.predict(X_test)



<h2 id=eval>Evaluación de la Performance del Regresor</h2>

Evaluaremos la performance del regresor para el dataset de testing a traves de las métricas ya vistas en el curso.
Si bien calcularemos todas las metricas usuales nos guiaremos por el score F1 el cual es la media armonina entre la precision y el recall.

In [ ]:
# Evaluate model performance

cm = confusion_matrix(y_test, y_pred)
tp = cm[1, 1]  # True Positives
fp = cm[0, 1]  # False Positives
fn = cm[1, 0]  # False Negatives
tn = cm[0, 0]  # True Negatives

# Accuracy
accuracy = (tp + tn) / (tp + tn + fp + fn)
# Precision
precision = tp / (tp + fp)
# Recall
recall = tp / (tp + fn)
# Specificity
specificity = tn / (tn + fp)
# F1 Score
f1 = 2 * (precision * recall) / (precision + recall)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('Specificity:', specificity)
print('F1 score:', f1)

Podemos Plotear la matriz de confusión tal como lo vimos en las slides. 
Como ejercicio les dejo de ejercicio hacer el grafico de la matriz de confusion en colores tal como aparece en el notebook "ML0101EN-Clas-SVM-cancer.ipyn"

Hagamos algunas predicciones

In [ ]:
X_eval=features.sample(10).values 

In [ ]:
preds = model.predict(X_eval)
print(X_eval, preds)

## Validación del Modelo
Genial, hemos creado un modelo de regresion logistica, lo hemos entrenado y hemos validado su preformance con el conjunto de datos de testing. También hemos calculado las metricas para evaluar la bondad del modelo.
Ahora la gran pregunta, el modelo esta validado?

Podriamos sentirnos tentados a decir que si ya que el F1 score de 0.78 no es mal valor. Pero en general solo podemos decir que el modelo es bueno. que predice aceptablemnte si un estudiante egresará en función de los créditos ganados a los 3 años.

Una forma mas profesional de validar el Modelo seria calcular el F1 score para prediciones hechas con un conjunto de datos completamente direferente del de test y ver que pasa.

En caso que los valores de F1 se mantengan cercanos podremos decir que el modelo es relativamente robusto.Y en caso negativo deberíamos evaluar otras posibilidades, las cuales les dejo para pensar.

Otra posibilidad es usr la columns creditos a 6 años  junto con creditos a 3 años como caracteristica a ver si metiendo esta otra columna y repitinedo todos los pasos anteriores mejora la performance. (lo dejo como ejercicio)

In [ ]:
with engine.connect() as conn:
    rs = conn.execute('delete from temporal')
    conn.close()

query4 = 'insert into temporal select distinct cedula from estudcarr where generacion between 2010 and 2012 \
        and carrera in (22, 72)' 

# Lleno la tabla tremporal con las cedulas deseadas

with engine.connect() as conn:
    rs = conn.execute(query1)
    conn.close()




In [ ]:
# Nvamente Saco la suma de créditos a los tres años de haber empezado.

query2 = 'select * from (select s.cedula as "cedula", u.carrera as "carrera", u.ciclo as "ciclo", sum(creditos) as "Creds3a", \
Fechaing from activ2 s, asigcarr t, estudcarr u \
where s.cedula = u.cedula and t.carrera = u.carrera and t.ciclo = u.ciclo and t.carrera in (22, 72) and \
s.asignatura= t.asignatura and tipoactividad in ("E", "R") and nota > 2 and fecha between Fechaing and \
Fechaing + 30300 and s.cedula in temporal \
group by u.cedula, u.carrera, u.ciclo) group by cedula having(max(Creds3a))'


df = pd.read_sql_query(query2, engine)

Ejercicio: Rehacer todos los comandos a partir de la exploracion de datos y Preparando los datos de entrada  con este nvo dataset hacer predicciones y calcular las metricas en especial F1 (No crear el modelo , ni entrenarlo porque ya lo esta)

<h2 id=fin>Conclusiones</h2>

Felicitaciones! han llegado al final de este ejercicio, espreo que les haya servido para repasar algunso conceptos que vimos. 

### Autor

<a href='https://www.linkedin.com/in/ram%C3%B3n-c-4389b6b/' >Ram&oacute;n Caraballo</a> Abril 2025